In [1]:
#surrogate models
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split

data_file = './all_parts_vectors_drags.csv'
df = TabularDataset(data_file)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=777)

#exclue the first two columns of train data
train_data = train_df.drop(columns=['i', 'name'])
train_data.head()

/home/xli/anaconda3/envs/surrogate/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_440,dim_441,dim_442,dim_443,dim_444,dim_445,dim_446,dim_447,dim_448,drag
61,-6.639910,-2.575970,-4.333867,0.348634,-6.591602,4.137893,3.448946,9.801783,-4.944950,-1.340411,...,-1.080956,0.063173,1.612214,1.418663,0.173674,0.923131,-0.423641,-2.278858,-0.536435,0.375
354,5.044733,2.543681,-3.247805,-5.378260,0.549456,-1.075264,6.259791,5.443143,1.828774,5.774058,...,-1.829254,2.322891,0.782790,1.487128,-1.055578,2.655854,-0.162912,-0.385024,0.673573,0.374
358,-0.939149,0.704528,0.749587,-1.643254,-3.556550,0.293420,3.039790,-3.084175,-3.810488,3.759489,...,-0.636529,10.484594,10.753080,8.283512,1.325511,6.276447,-0.932776,11.217846,1.199296,0.435
275,-1.177986,1.175732,-1.834498,1.306601,-0.493694,1.298445,2.197183,4.541283,1.859666,3.284920,...,-2.321944,1.785326,0.453086,0.186109,1.353902,2.105992,-0.312803,-1.399713,0.909484,0.437
18,0.390329,-1.530953,-2.521950,-4.864147,-0.742480,0.146031,-0.156536,1.740131,-2.943921,5.280754,...,-2.205936,-0.899805,0.860134,1.484055,-2.426541,2.012407,-0.914859,-1.245621,0.841827,0.367


In [2]:
label = 'drag'
print("Summary of class variable: \n", train_data[label].describe())

Summary of class variable: 
 count    351.000000
mean       0.398513
std        0.060013
min        0.278000
25%        0.353000
50%        0.394000
75%        0.435000
max        0.598000
Name: drag, dtype: float64


In [3]:
import os
save_path = './agModels-all_parts'  # specifies folder to store trained models
if not os.path.exists(save_path):
    os.makedirs(save_path)

metric = 'root_mean_squared_error' #Regression:mean_absolute_error, mean_squared_error,root_mean_squared_error (default), r2
predictor = TabularPredictor(label=label, path=save_path, eval_metric=metric).fit(train_data, presets='best_quality', num_bag_folds=0, num_bag_sets=1)

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=0, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "./agModels-all_parts/"
AutoGluon Version:  0.7.0
Python Version:     3.10.10
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #51-Ubuntu SMP Mon Jul 4 06:41:22 UTC 2022
Train Data Rows:    351
Train Data Columns: 448
Label Column: drag
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (0.598, 0.278, 0.39851, 0.06001)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory: 

In [4]:
test_data = test_df.drop(columns=['i', 'name'])
# val_data.head()
y_val = test_data[label]
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_439,dim_440,dim_441,dim_442,dim_443,dim_444,dim_445,dim_446,dim_447,dim_448
46,3.262418,4.623484,5.207058,1.474604,5.149589,1.366774,7.350364,4.988407,-4.451728,2.715955,...,0.493204,-1.530669,-1.038711,1.546212,0.701456,-0.417296,2.205014,-2.338213,-1.252558,-0.207984
101,-1.815653,0.467063,-0.713573,-2.519432,-6.833046,3.925573,3.131740,2.785505,-0.169789,3.797550,...,-1.799442,-2.883593,1.992778,0.544198,1.757790,-0.963274,2.303173,-0.294457,-0.247084,2.795963
175,-0.128397,-0.347771,-4.491439,-0.143762,-1.139696,1.523660,4.283424,1.275272,-3.277825,-0.944104,...,-0.005584,-2.087350,0.265765,1.710699,-0.205781,0.221282,2.097013,-2.484610,-1.032043,0.472173
9,-0.334880,-0.358754,-5.121314,-1.869275,0.053525,2.998930,-0.482418,1.283854,-1.355309,-0.988547,...,2.702447,-1.539050,-0.802389,1.228584,1.375461,-2.406342,0.712337,-1.958422,-1.374814,-1.371111
136,0.339348,2.649365,-1.346519,0.872298,-2.672357,1.983254,2.075392,6.573480,-0.809723,-2.231186,...,4.364009,-0.522479,0.352676,0.769096,1.184941,2.701294,0.989609,-2.073087,-1.652956,-0.242727


In [6]:
%%capture log_output
%config InlineBackend.figure_format = 'retina'
%config Application.log_level = 'DEBUG'
%config IPCompleter.greedy = True

predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file
y_pred = predictor.predict(test_data_nolab)
for item in y_pred:
    print(item)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_val, y_pred=y_pred, auxiliary_metrics=True)
print(perf)

results = predictor.fit_summary(show_plot=True)
print(results)
print(predictor.leaderboard(test_data, silent=True))

with open('./output_all_parts.log', 'w') as f:
    f.write(log_output.stdout)

Evaluation: root_mean_squared_error on test data: -0.049616523059487175
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "root_mean_squared_error": -0.049616523059487175,
    "mean_squared_error": -0.0024617993605126226,
    "mean_absolute_error": -0.03747519815239039,
    "r2": 0.16282727788866536,
    "pearsonr": 0.41014733083055,
    "median_absolute_error": -0.032015099525451673
}


In [ ]:
print("AutoGluon infers problem type is: ", predictor.problem_type)
print("AutoGluon identified the following types of features:")
print(predictor.feature_metadata)

AutoGluon infers problem type is:  regression
AutoGluon identified the following types of features:
('float', []) : 64 | ['dim_1', 'dim_2', 'dim_3', 'dim_4', 'dim_5', ...]


In [7]:
train_data_pred = predictor.predict(train_data, model='WeightedEnsemble_L2')
for item in train_data_pred:
    print(item)

0.37861838936805725
0.38249272108078003
0.4381767511367798
0.43621888756752014
0.36622968316078186
0.366889089345932
0.41771551966667175
0.41200411319732666
0.36638909578323364
0.3539412021636963
0.34941190481185913
0.508400559425354
0.4613805115222931
0.37091490626335144
0.42514297366142273
0.4128950238227844
0.30657824873924255
0.4202014207839966
0.49697691202163696
0.35575976967811584
0.3942566215991974
0.46309420466423035
0.3245896100997925
0.43436768651008606
0.4211915135383606
0.4247615933418274
0.3925668001174927
0.4259239435195923
0.32838764786720276
0.3498603105545044
0.4584028124809265
0.42842620611190796
0.491293728351593
0.4479084610939026
0.3599356710910797
0.37358254194259644
0.4128498136997223
0.40936943888664246
0.3293745517730713
0.34855908155441284
0.49886441230773926
0.37742623686790466
0.33937087655067444
0.31430166959762573
0.3420461118221283
0.41306015849113464
0.3899974226951599
0.40397006273269653
0.3414789140224457
0.39428237080574036
0.395708829164505
0.362229

In [8]:
test_data_pred = predictor.predict(test_data, model='WeightedEnsemble_L2')
for item in test_data_pred:
    print(item)

0.37111416459083557
0.3560630679130554
0.4008951783180237
0.41535258293151855
0.4331023097038269
0.37909644842147827
0.398598313331604
0.4546825885772705
0.4231151342391968
0.36642080545425415
0.39534491300582886
0.39965587854385376
0.3610095977783203
0.42284712195396423
0.4054073095321655
0.41756123304367065
0.3687572479248047
0.4054275155067444
0.37956365942955017
0.36853331327438354
0.41840681433677673
0.4042297601699829
0.3840467929840088
0.38012588024139404
0.43543678522109985
0.39734330773353577
0.44037193059921265
0.3608059883117676
0.3968009352684021
0.37854433059692383
0.4255990982055664
0.39983874559402466
0.4111539125442505
0.4331341087818146
0.37638670206069946
0.36026841402053833
0.3626856207847595
0.41154545545578003
0.3939424753189087
0.3762201964855194
0.455719918012619
0.37719839811325073
0.42263007164001465
0.37848570942878723
0.3750516176223755
0.4030238389968872
0.39069393277168274
0.35904914140701294
0.400773823261261
0.40457651019096375
0.42658737301826477
0.34091